## Create the figures used in Stochastic Variance Reduced Gradient Plug-and-Play Methods for Solving Image Reconnstruction Problems

In [1]:
%matplotlib notebook

from imports import *

print('cuda' if torch.cuda.is_available() else 'cpu')

Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
                 It can be downloaded at https://aka.ms/vs/16/release/vc_redist.x64.exe
cuda


# Figures 1 + 2: Compare Image Fidelity for Different Algorithms and Compare PSNR over time using NLM

In [ ]:
res = create_problem()

In [ ]:
z_SVRG, time_SVRG, psnr_SVRG, zs_SVRG = pnp_svrg(res, 'nlm', 2000, 30, 20, 1000, True)

In [ ]:
z_GD, time_GD, psnr_GD, zs_GD = pnp_gd(res, 'nlm', 2000, 600, True)

In [ ]:
z_SGD, time_SGD, psnr_SGD, zs_SGD = pnp_sgd(res, 'nlm', 2000, 600, 10000, True)

In [ ]:
z_LSVRG, time_LSVRG, psnr_LSVRG, zs_LSVRG = pnp_lsvrg(res, 'nlm', 800, 600, 10000, 0.05, True)

In [ ]:
show_grid([torch.Tensor(res['original']), torch.Tensor(res['noisy']), 
           torch.Tensor(z_SVRG), torch.Tensor(z_GD), torch.Tensor(z_SGD), torch.Tensor(z_LSVRG)], figsize=(8,8))
plt.savefig("test.png",bbox_inches='tight')

In [ ]:
# Figure to display iteration count/ clock time vs PSNR for all algorithms
psnr_fig = plt.figure()
psnr_ax = psnr_fig.add_subplot(1, 1, 1)

psnr_ax.plot(np.cumsum(time_SVRG), psnr_SVRG)
psnr_ax.plot(np.cumsum(time_GD), psnr_GD)
psnr_ax.plot(np.cumsum(time_SGD), psnr_SGD)
psnr_ax.plot(np.cumsum(time_LSVRG), psnr_LSVRG)

psnr_ax.set(xlabel='time (s)', ylabel='PSNR (dB)')
psnr_ax.legend(('PnP-SVRG', 'PnP-GD', 'PnP-SGD', 'PnP-LSVRG'), loc='lower right')
psnr_ax.grid()

psnr_fig.tight_layout()

plt.show()

## Figure 3: Compare Image Denoising Routines with PnP-SVRG

In [2]:
res = create_problem()

In [ ]:
z_SVRG_nlm, time_SVRG_nlm, psnr_SVRG_nlm, zs_SVRG_nlm = pnp_svrg(res, 'nlm', 2000, 30, 20, 1000, True)

In [ ]:
z_SVRG_tv, time_SVRG_tv, psnr_SVRG_tv, zs_SVRG_tv = pnp_svrg(res, 'tv', 2000, 30, 20, 1000, True)

In [ ]:
z_SVRG_bm3d, time_SVRG_bm3d, psnr_SVRG_bm3d, zs_SVRG_bm3d = pnp_svrg(res, 'bm3d', 2000, 30, 20, 1000, True)

In [ ]:
z_SVRG_cnn, time_SVRG_cnn, psnr_SVRG_cnn, zs_SVRG_cnn = pnp_svrg(res, 'cnn', 1000, 30, 20, 1000, True)

In [ ]:
show_grid([torch.Tensor(res['original']), torch.Tensor(res['noisy']), 
           torch.Tensor(z_SVRG), torch.Tensor(z_GD), torch.Tensor(z_SGD), torch.Tensor(z_LSVRG)], figsize=(8,8))

## Figure 4: Compare Sampling Ratio for each PnP Algorithm

In [ ]:
num_probs = 100
num_trials = 10
sample_probs = np.linspace(0.2, 1, num_probs)

prob_vs_psnr_SVRG = []
prob_vs_psnr_GD = []
prob_vs_psnr_SGD = []
prob_vs_psnr_LSVRG = []

for i in range(num_probs):
    samp_ratio_SVRG = []
    samp_ratio_GD = []
    samp_ratio_SGD = []
    samp_ratio_LSVRG = []
    for j in range(num_trials):
        MASK = generate_mask(height=256, width=256, sample_prob=sample_probs[i])
        res = create_problem(img_path='./data/Set12/13.jpg', sigma=1.0, mask_=MASK)
        denoise_parameters = nlm_config()

        z_SVRG, time_SVRG, psnr_SVRG, zs_SVRG = pnp_svrg(res, 'nlm', denoise_parameters, 0.095, 10, 10, 10000, False)
        samp_ratio_SVRG.append(psnr_SVRG[-1])

        z_GD, time_GD, psnr_GD, zs_GD = pnp_gd(res, 'nlm', denoise_parameters, 0.8, 100, False)
        samp_ratio_GD.append(psnr_GD[-1])

        z_SGD, time_SGD, psnr_SGD, zs_SGD = pnp_sgd(res, 'nlm', denoise_parameters, 0.8, 100, 10000, False)
        samp_ratio_SGD.append(psnr_SGD[-1])

        z_LSVRG, time_LSVRG, psnr_LSVRG, zs_LSVRG = pnp_lsvrg(res, 'nlm', denoise_parameters, 0.095, 10, 10000, 0.1, False)
        samp_ratio_LSVRG.append(psnr_LSVRG[-1])
    prob_vs_psnr_SVRG.append(np.average(samp_ratio_SVRG))
    prob_vs_psnr_GD.append(np.average(samp_ratio_GD))
    prob_vs_psnr_SGD.append(np.average(samp_ratio_SGD))
    prob_vs_psnr_LSVRG.append(np.average(samp_ratio_LSVRG))
# PLOT CURVES